# python のデコレーター構文チュートリアル

本文章は筆者がデコレータを学ぶために,  http://www.nasuinfo.or.jp/FreeSpace/kenji/sf/python/virtualMachine/decorator.htm にある文章を `jupyter-notebook` で実行したものになっている.
また, 参考にした文書のコードがPython2対応のものとなっており, 若干古いため, Python3で動くように修正した. また所々デコレータを用いずに等価な処理を記述している. 

<s>別にこの文章は誰かに公開するつもりでもなく, ただself-educationのために記す文章である. </s>

@decorate 構文での syntax sugar とは「関数を置き換える記述を簡潔に表す」構文を意味しています。closure を使わない、関数を入れ替えるだけの意味を強調した次のようなコードを書けます。

In [2]:
def test(func):
    print("Now, here is in test.")
    print("this is called from {}".format(func.__name__))
    return func

@test
def deco():
    print("I'm deco.")

print('------------')
deco()

Now, here is in test.
this is called from deco
------------
I'm deco.


これはQiitaにあったサンプルコードで, これをみても何もわからなかった.

In [20]:
def dcrtF(objAg):
    print("Now in dcrtF(.):",objAg)
    return "string returned by dcrtF(.) function"

@dcrtF
def testF( objAg ):
    print("Now in testF(.):", objAg)

print("--------------")
print(testF) # testF is string, not function
type(testF)

Now in dcrtF(.): <function testF at 0x10b8e2e18>
--------------
string returned by dcrtF(.) function


str

上では `testF` 関数を `@dcrtF` 構文によって修飾しています。`@dcrtF` を読んだ `python` インタプリタは `dcrtF(.)` 関数を呼び出します。`dcrtF(.)` 関数の引数値は `testF` 関数オブジェクトです。そして `dcrtF(.)` 関数の戻り値：`"e;;string returned by dcrtF(.) function";` 文字列オブジェクトで `testF` 関数を置き換えます。`@dcrtF` 修飾によって `testF` はラベルは文字列オブジェクトを指すようになってしまいました。関数ではなくなってしまいました。
上のように `@dcrtF` 構文で引数が存在しないとき、修飾される関数 `testF` オブジェクトを `dcrtF(.)` 関数の引数として与えます。上の例では `dcrtF(testF)` と呼び出します。もし `dcrtF()` 関数が引数を一つも持たない関数のときは「`TypeError: dcrtF() takes no arguments (1 given)`」と コンパイル・エラーになります。

実際に `type(testF)` を実行すると, `testF` が文字列オブジェクトになっていることがわかります. 

通常の実用的なデコレーター構文では、`dcrtF` 関数は `callable object` を返します。例えば `testF` ラベルの指し示す先を別の関数オブジェクトにすることで、デコレーター構文での修飾の後でも `testF(.)` 呼び出しが可能にしておきます。

In [9]:
def testF2( objAg ):
    print("Now in testF2(.):", objAg)

def dcrtF(fnAg):
    print("Now in dcrtF(.):",fnAg)
    return testF2

@dcrtF
def testF( objAg ):
    print ("Now in testF(.):", objAg)

print("--------------")
testF(5)

Now in dcrtF(.): <function testF at 0x1039ba048>
--------------
Now in testF2(.): 5


上のコード例では `testF` を `testF2` に入れ替えたので `testF(.)` 呼び出しが `testF2(.)` 呼び出しに代わっています。
python のデコレーター構文では、通常は引数として渡された関数に 別の機能を追加した関数を返すことで関数を修飾します。そのために `dcrtF(.)` の引数は、修飾される関数オブジェクトになっているわけです。今度は python の decorator らしく渡される関数オブジェクトに機能を追加した関数オブジェクトを返します。具体的には関数の実行時間を計測する機能で修飾してみます。今までの python decorator の syntax sugar が理解できていれば難しくありません。まだ関数の置き換えにすぎません。 closure 機能を使いません。

In [10]:
fnBufferStt=None
def testF2( objAg ):
    import time
    print("Now in testF2(.) and start testF(objAg):", objAg)
    dbStartTimeAt = time.clock()
    fnBufferStt(objAg)
    print(time.clock() - dbStartTimeAt)

def dcrtF(fnAg):
    global fnBufferStt
    print("Now in dcrtF(.):",fnAg)
    fnBufferStt = fnAg
    return testF2

@dcrtF
def testF( objAg ):
    print ("Now in testF(.):", objAg, "   sum up:", sum(range(1000)))

print("--------------")
testF(5)

Now in dcrtF(.): <function testF at 0x1039ba620>
--------------
Now in testF2(.) and start testF(objAg): 5
Now in testF(.): 5    sum up: 499500
0.00018099999999998673


上のコードでは `@dcrtF` の位置で python interpreter が渡した `testF` 関数オブジェクトを `fnBufferStt` に保存しています。後で使うためです。そして `testF2` と `testF` を入れ替えます。すなわち 最後の行で `testF(5)` と `testF` を実行したつもりでも `testF2(5)` が実行されるようになります。そして `testF2(5)` を実行するときに `fnBufferStt` に保存してあった `testF` 関数を引数値：5で呼び出します。その呼び出しの前後で、プログラムを起動してからの経過時間を `time.clock()` で計って その差分を計算することにより `testF(5)` の実行時間が計測できます。

上のコードは以下のコードと等価である. 

In [12]:
fnBufferStt=None
def testF2( objAg ):
    import time
    print("Now in testF2(.) and start testF(objAg):", objAg)
    dbStartTimeAt = time.clock()
    fnBufferStt(objAg)
    print(time.clock() - dbStartTimeAt)

def dcrtF(fnAg):
    global fnBufferStt
    print("Now in dcrtF(.):",fnAg)
    fnBufferStt = fnAg
    return testF2

def testF( objAg ):
    print ("Now in testF(.):", objAg, "   sum up:", sum(range(1000)))

testF = dcrtF(testF)

print("--------------")
testF(5)

Now in dcrtF(.): <function testF at 0x1039a1620>
--------------
Now in testF2(.) and start testF(objAg): 5
Now in testF(.): 5    sum up: 499500
0.0002449999999998287


ここでの `@dcrF` による修飾は任意の関数に適用できます。任意の関数の実行時間を計測できます。ただし一つの関数しかデコレートできません。 `fnBufferStt` が一つしかないからです。別の関数をデコレートすると `fnBufferStt` が その別の関数オブジェクトに入れ替わってしまうからです。このような問題を回避できるように `closure` 機能を使います。

## デコレーターの closure 機能

dcrtF(.) 関数の内側に関数をネストさせることで、上の fnBufferStt グローバル変数では一つの関数しか修飾できない問題を奇麗に回避できます。これが closure 機能を利用したことになっています。

In [2]:
def dcrtF(fnAg):
    print("Now in dcrtF(.):",fnAg)

    def innerF(objAg):
        import time
        print ("Now innerF(.)", objAg)
        dbStartTimeAt = time.clock()
        fnAg(objAg)
        print (time.clock() - dbStartTimeAt)

    return innerF

@dcrtF
def sumUp( inAg ):
    print ("Now in sumUp(.):", inAg, "   sum up:", sum(range(inAg)))

import numpy as sc
@dcrtF
def sumUpByNum( inAg ):
    #import numarray as sc
    print("Now in testFF(.):", inAg, "   sum up:", sum(sc.arange(inAg)))

print ("--------------")
sumUp(10)
print ("--------------")
sumUpByNum(10)
print ("--------------")
sumUp(100000)
print ("--------------")
sumUpByNum(100000)

Now in dcrtF(.): <function sumUp at 0x1090ba730>
Now in dcrtF(.): <function sumUpByNum at 0x10a808950>
--------------
Now innerF(.) 10
Now in sumUp(.): 10    sum up: 45
0.0004889999999999617
--------------
Now innerF(.) 10
Now in testFF(.): 10    sum up: 45
0.0006490000000001217
--------------
Now innerF(.) 100000
Now in sumUp(.): 100000    sum up: 4999950000
0.00287000000000015
--------------
Now innerF(.) 100000
Now in testFF(.): 100000    sum up: 4999950000
0.013651999999999997


### callable オブジェクトによる closure オブジェクトの代用

ここまで読んで、「 closure ではなく callable な class instance でも同じことができるじゃないか」と思った方は鋭い感性をお持ちです。そのとおりです。「コード例 4」の関数実行時間の計測機能のデコレーションは、下のクラス・インスタンスで作った callable object を使ったデコレーションでも実現できます。

callableなオブジェクトが返って来ればいいなら, クラスでも同じことができる. 

In [12]:
class ClMesureTime:
    def __init__(self, fnObjAg):
        self.m_fnObj = fnObjAg
    
    def __call__(self, objAg):
        import time
        print("Now ClMesureTime.__call__(.)", self.m_fnObj)
        dbStartTimeAt = time.clock()
        self.m_fnObj(objAg)
        print(time.clock() - dbStartTimeAt)
        

def dcrtF(fnAg):
    print ("Now in dcrtF(.):",fnAg)
    return ClMesureTime(fnAg)

@dcrtF
def sumUp( inAg ):
    print ("Now in sumUp(.):", inAg, "   sum up:", sum(range(inAg)))

import numpy as sc
@dcrtF
def sumUpByNum( inAg ):
    #import numarray as sc
    print ("Now in testFF(.):", inAg, "   sum up:", sum(sc.arange(inAg)))

print("--------------")
sumUp(10)
print("--------------")
sumUpByNum(10)
print("--------------")
sumUp(100000)
print("--------------")
sumUpByNum(100000)

Now in dcrtF(.): <function sumUp at 0x10b8c3840>
Now in dcrtF(.): <function sumUpByNum at 0x10b8bcd08>
--------------
Now ClMesureTime.__call__(.) <function sumUp at 0x10b8c3840>
Now in sumUp(.): 10    sum up: 45
0.00015900000000002024
--------------
Now ClMesureTime.__call__(.) <function sumUpByNum at 0x10b8bcd08>
Now in testFF(.): 10    sum up: 45
0.00025000000000030553
--------------
Now ClMesureTime.__call__(.) <function sumUp at 0x10b8c3840>
Now in sumUp(.): 100000    sum up: 4999950000
0.0032529999999999504
--------------
Now ClMesureTime.__call__(.) <function sumUpByNum at 0x10b8bcd08>
Now in testFF(.): 100000    sum up: 4999950000
0.011570999999999998


この場合も, 上と等価なコードは以下のようになる. 

In [13]:
class ClMesureTime:
    def __init__(self, fnObjAg):
        self.m_fnObj = fnObjAg
    
    def __call__(self, objAg):
        import time
        print("Now ClMesureTime.__call__(.)", self.m_fnObj)
        dbStartTimeAt = time.clock()
        self.m_fnObj(objAg)
        print(time.clock() - dbStartTimeAt)
        

def dcrtF(fnAg):
    print ("Now in dcrtF(.):",fnAg)
    return ClMesureTime(fnAg)


def sumUp( inAg ):
    print ("Now in sumUp(.):", inAg, "   sum up:", sum(range(inAg)))
sumUp = dcrtF(sumUp)
    
import numpy as sc

def sumUpByNum( inAg ):
    #import numarray as sc
    print ("Now in testFF(.):", inAg, "   sum up:", sum(sc.arange(inAg)))
sumUpByNum = dcrtF(sumUpByNum)

print("--------------")
sumUp(10)
print("--------------")
sumUpByNum(10)
print("--------------")
sumUp(100000)
print("--------------")
sumUpByNum(100000)

Now in dcrtF(.): <function sumUp at 0x10a808950>
Now in dcrtF(.): <function sumUpByNum at 0x10b8c31e0>
--------------
Now ClMesureTime.__call__(.) <function sumUp at 0x10a808950>
Now in sumUp(.): 10    sum up: 45
0.0003959999999998409
--------------
Now ClMesureTime.__call__(.) <function sumUpByNum at 0x10b8c31e0>
Now in testFF(.): 10    sum up: 45
0.000568000000000346
--------------
Now ClMesureTime.__call__(.) <function sumUp at 0x10a808950>
Now in sumUp(.): 100000    sum up: 4999950000
0.00293499999999991
--------------
Now ClMesureTime.__call__(.) <function sumUpByNum at 0x10b8c31e0>
Now in testFF(.): 100000    sum up: 4999950000
0.011724000000000068


ただし別の [dis/inspect モジュールを使った Python のハッキング](http://www.nasuinfo.or.jp/FreeSpace/kenji/sf/python/virtualMachine/PyVM.htm)で詳しく説明するように closure のほうが単純な python virtual machine コードになります。効率的なコードになります。callable なクラス・インスタンスを設けるには class の生成とクラスに付属する `dict` 変数を生成せねばなりません。closure ならば `innerF` 関数オブジェクトに `inAg` 変数を追加するだけで済みます。

### @decorate 構文での引数

@decorate 構文には引数を与えられます。より汎用的な修飾が可能です。ただし引数を与えられたときは、`decorate(引数)` の戻り値が callable であることを前提に、一つの `@decorate(.)` 構文に対し、二回の関数呼び出しが発生します。 一回目の `decorate(引数)` が戻した callable オブジェクトに対して、修飾される関数を引数として渡して二回目の呼び出しを行います。二回目の関数呼び出しの戻り値で修飾される関数を置き換えます。

以下の例では, `@dtrtF(strComment)` とコメント文字列を引き渡せるようにしたテスト・コードを使います。

In [22]:
# decoratorを使って書く
class ClMesureTime:
    def __init__(self, strCommentAg):
        self.m_strComment = strCommentAg
    
    def __call__(self, fnObjAg):
        print("Now ClMesureTime.__call__(.)", fnObjAg)
        self.m_fnObj = fnObjAg
        return self.__decorate

    def __decorate(self, objAg):
        import time
        print("Now __decorate(.):"+self.m_strComment+": ", objAg)
        dbStartTimeAt = time.clock()
        self.m_fnObj(objAg)
        print(time.clock() - dbStartTimeAt)


def dcrtF(strCommentAg):
    print("Now in dcrtF(.):",strCommentAg)
    return ClMesureTime(strCommentAg)


@dcrtF("sumUp function")
def sumUp( inAg ):
    print("Now in sumUp(.):", inAg, "   sum up:", sum(range(inAg)))

import numpy as sc
@dcrtF("sumUpByNum function")
def sumUpByNum( inAg ):
    #import numarray as sc
    print("Now in sumUpByNum(.):", inAg, "   sum up:", sum(sc.arange(inAg)))

print("--------------")
sumUp(10)
print("--------------")
sumUpByNum(10)
print("--------------")
sumUp(100000)
print("--------------")
sumUpByNum(100000)

Now in dcrtF(.): sumUp function
Now ClMesureTime.__call__(.) <function sumUp at 0x10b8d7a60>
Now in dcrtF(.): sumUpByNum function
Now ClMesureTime.__call__(.) <function sumUpByNum at 0x10b8d7158>
--------------
Now __decorate(.):sumUp function:  10
Now in sumUp(.): 10    sum up: 45
0.00040000000000040004
--------------
Now __decorate(.):sumUpByNum function:  10
Now in sumUpByNum(.): 10    sum up: 45
0.0008619999999996963
--------------
Now __decorate(.):sumUp function:  100000
Now in sumUp(.): 100000    sum up: 4999950000
0.0025800000000000267
--------------
Now __decorate(.):sumUpByNum function:  100000
Now in sumUpByNum(.): 100000    sum up: 4999950000
0.010448000000000235


`dcrtF(.)` 関数が の callble class instance を返しています。`@dcrtF(strCommentAg)` によるデコレーションで 一回目の `dcrtF(strCommentAg)` 関数の呼び出しに続いて、`dctrF(.)` 関数の戻り値 `ClMesureTime callable instance` による `sumUp`, `sumUpByNum` 関数オブジェクトを引数とする二回目の呼び出しを行っていることが解るはずです。二回目の呼び出しでは、関数の実行時間を計測するメンバー関数を返すので、それで `sumUp`, `sumUpByNum` 関数が置き換えられます。

これは以下に示すコードと等価です. 

In [23]:
# decoratorを使わないとこうなる
class ClMesureTime:
    def __init__(self, strCommentAg):
        self.m_strComment = strCommentAg
    
    def __call__(self, fnObjAg):
        print("Now ClMesureTime.__call__(.)", fnObjAg)
        self.m_fnObj = fnObjAg
        return self.__decorate

    def __decorate(self, objAg):
        import time
        print("Now __decorate(.):"+self.m_strComment+": ", objAg)
        dbStartTimeAt = time.clock()
        self.m_fnObj(objAg)
        print(time.clock() - dbStartTimeAt)


def dcrtF(strCommentAg):
    print("Now in dcrtF(.):",strCommentAg)
    return ClMesureTime(strCommentAg)



def sumUp( inAg ):
    print("Now in sumUp(.):", inAg, "   sum up:", sum(range(inAg)))
tmpobj_0 = dcrtF("sumUp function")
sumUp = tmpobj_0(sumUp)
    
import numpy as sc

def sumUpByNum( inAg ):
    #import numarray as sc
    print("Now in sumUpByNum(.):", inAg, "   sum up:", sum(sc.arange(inAg)))
tmpobj_1 = dcrtF("sumUpByNum function")
sumUpByNum = tmpobj_1(sumUpByNum)

print("--------------")
sumUp(10)
print("--------------")
sumUpByNum(10)
print("--------------")
sumUp(100000)
print("--------------")
sumUpByNum(100000)

Now in dcrtF(.): sumUp function
Now ClMesureTime.__call__(.) <function sumUp at 0x10a808950>
Now in dcrtF(.): sumUpByNum function
Now ClMesureTime.__call__(.) <function sumUpByNum at 0x10b8e2158>
--------------
Now __decorate(.):sumUp function:  10
Now in sumUp(.): 10    sum up: 45
0.0004679999999996909
--------------
Now __decorate(.):sumUpByNum function:  10
Now in sumUpByNum(.): 10    sum up: 45
0.000560000000000116
--------------
Now __decorate(.):sumUp function:  100000
Now in sumUp(.): 100000    sum up: 4999950000
0.002183000000000046
--------------
Now __decorate(.):sumUpByNum function:  100000
Now in sumUpByNum(.): 100000    sum up: 4999950000
0.010070999999999941


上記の処理を, 今度はクラスを用いずに, 関数で書くと以下のようになる. 3重のネストになっているため読みにくいかもしれないが, 1行ずつコードを追っていけば,  同じことをしているのがわかる. 

In [16]:
def dcrtF(strCommentAg):
    print("Now in dcrtF(.):",strCommentAg)

    def innerF(fnObjAg):
        def innerInnerF(objAg):
            import time
            print("Now innerF(.):"+strCommentAg+": ", objAg)
            dbStartTimeAt = time.clock()
            fnObjAg(objAg)
            print(time.clock() - dbStartTimeAt)

        return innerInnerF

    return innerF

@dcrtF("sumUp function")
def sumUp( inAg ):
    print("Now in sumUp(.):", inAg, "   sum up:", sum(range(inAg)))

import numpy as sc
@dcrtF("sumUpByNum function")
def sumUpByNum( inAg ):
    print("Now in testFF(.):", inAg, "   sum up:", sum(sc.arange(inAg)))

print("--------------")
sumUp(10)
print("--------------")
sumUpByNum(10)
print("--------------")
sumUp(100000)
print("--------------")
sumUpByNum(100000)

Now in dcrtF(.): sumUp function
Now in dcrtF(.): sumUpByNum function
--------------
Now innerF(.):sumUp function:  10
Now in sumUp(.): 10    sum up: 45
0.0007629999999996251
--------------
Now innerF(.):sumUpByNum function:  10
Now in testFF(.): 10    sum up: 45
0.0005589999999999762
--------------
Now innerF(.):sumUp function:  100000
Now in sumUp(.): 100000    sum up: 4999950000
0.0036969999999998393
--------------
Now innerF(.):sumUpByNum function:  100000
Now in testFF(.): 100000    sum up: 4999950000
0.012243999999999922


上の例では, `@dcrtF(strCommentAg)` の `strCommentAg` 引数を `self.m_strComment` などに保存する手間をかけずに、内側の関数の外側関数の変数参照で `innerInnerF` closure に保存されています。

より実用的なデコレータの例として, 以下のようなものがある.

In [18]:
import math

#def declareArgs():
def declareArgs(*argTypes):

    def checkArguments(func):   # @declareArgs() のときに働く
        assert func.__code__.co_argcount == len(argTypes)

        def wrapper(*args, **kwargs):
            pos = 1
            for (arg, argType) in zip(args, argTypes):
                assert isinstance(arg, argType), \
                        "Value %r dose not match %s at %d" % (arg, argType, pos)
                pos += 1
            return func(*args, **kwargs)
        #wrapper.func_name = func.func_name 
        return wrapper
    #import dis
    #dis.dis(checkArguments)
    return checkArguments   # checkArgments 関数オブジェクトを返している

@declareArgs(float, float)
def calcDistance(x, y):
    return math.sqrt(x * x + y * y)

print("------------")
print(calcDistance(3.14, 1.592))
print("------------")
print(calcDistance(3.14, "abc"))

------------
3.5205204160748735
------------


AssertionError: Value 'abc' dose not match <class 'float'> at 2

`@declareArgs(float, float)` により、`calcDistance(.)` 関数の二つの引数のどちらも `float` タイプであることを確認するように修飾しました。ですから `calcDistance(3.14, 1.592)` を呼び出したたときは、正常に計算できました。でも `calcDistance(3.14, "abc")` を呼び出したときは `AssertionError` が発行されました. 